In [39]:
import subprocess
import random
import os
import pandas as pd
import numpy as np
import tqdm
import shutil

In [40]:
MACH = 0.5
MACH_VARIABILITY = 0.01

ALPHAS = [5, 6]
ALPHA_VARIABILITY = 0.1

NUM_SIMS = 100
BATCH_SIZE = 100

OUTPUT_FILE = "./outputs"

In [41]:
def run_xfoil_commands(command_list):
    """
    Run a batch of XFOIL commands using subprocess and return the output.
    """
    # Join the list of commands into a single string with each command separated by a newline
    commands = "\n".join(command_list)
    print(commands)
    
    process = subprocess.Popen(['./XFOIL6.99/xfoil'],
                               stdin=subprocess.PIPE,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               text=True
                               )

    output, error = process.communicate(input=commands)
    return output

In [42]:
def create_commands(alpha):
    output_path = f"./outputs/{alpha}"
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path)

    machs = []
    alphas = []

    machs = [round(random.uniform(MACH - MACH_VARIABILITY, MACH + MACH_VARIABILITY), 3) for _ in range(NUM_SIMS)]
    alphas = [round(random.uniform(alpha - ALPHA_VARIABILITY, alpha + ALPHA_VARIABILITY), 3) for _ in range(NUM_SIMS)]

    np.save(f"{output_path}/machs.npy", machs)
    np.save(f"{output_path}/alphas.npy", alphas)

    commands = [
        "PLOP",
        "G F",
        "", 
        "NACA 0012",
        "OPER",
        "ITER 400",
        "Re 250000",
        "VISC"
    ]

    for i in range(NUM_SIMS):
        new_commands = [
            f"MACH {machs[i]}",
            "PACC",
            f"{output_path}/{i}.txt",
            "",
            f"ALFA {alphas[i]}",
            "PACC",
            "PDEL",
            "0"
        ]
        commands.extend(new_commands)
    
    return commands

In [43]:
for alpha in ALPHAS:
    commands = create_commands(alpha)
    output = run_xfoil_commands(commands)

PLOP
G F

NACA 0012
OPER
ITER 400
Re 250000
VISC
MACH 0.505
PACC
./outputs/5/0.txt

ALFA 4.949
PACC
PDEL
0
MACH 0.504
PACC
./outputs/5/1.txt

ALFA 4.902
PACC
PDEL
0
MACH 0.497
PACC
./outputs/5/2.txt

ALFA 5.042
PACC
PDEL
0
MACH 0.499
PACC
./outputs/5/3.txt

ALFA 4.965
PACC
PDEL
0
MACH 0.498
PACC
./outputs/5/4.txt

ALFA 4.935
PACC
PDEL
0
MACH 0.51
PACC
./outputs/5/5.txt

ALFA 4.983
PACC
PDEL
0
MACH 0.507
PACC
./outputs/5/6.txt

ALFA 4.979
PACC
PDEL
0
MACH 0.496
PACC
./outputs/5/7.txt

ALFA 5.079
PACC
PDEL
0
MACH 0.509
PACC
./outputs/5/8.txt

ALFA 5.015
PACC
PDEL
0
MACH 0.496
PACC
./outputs/5/9.txt

ALFA 5.085
PACC
PDEL
0
MACH 0.506
PACC
./outputs/5/10.txt

ALFA 5.027
PACC
PDEL
0
MACH 0.509
PACC
./outputs/5/11.txt

ALFA 5.06
PACC
PDEL
0
MACH 0.498
PACC
./outputs/5/12.txt

ALFA 5.085
PACC
PDEL
0
MACH 0.506
PACC
./outputs/5/13.txt

ALFA 5.065
PACC
PDEL
0
MACH 0.503
PACC
./outputs/5/14.txt

ALFA 5.039
PACC
PDEL
0
MACH 0.508
PACC
./outputs/5/15.txt

ALFA 4.919
PACC
PDEL
0
MACH 0.492
PACC
./o

In [44]:
def read_output(file_path):
    with open(file_path, 'r') as file:
        for i, line in enumerate(file, start=1):
            if i == 13:  # Check if it's the 13th line
                values = line.split()
                alpha = float(values[0])
                cl = float(values[1])
                cd = float(values[2])
                cdp = float(values[3])
                cm = float(values[4])
                top_xtr = float(values[5])
                bot_xtr = float(values[6])
                return alpha, cl, cd, cdp, cm, top_xtr, bot_xtr
    return None  # Return None if the file doesn't have 13 lines

In [45]:
def process_folder(machs, alphas, folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            result = read_output(file_path)
            if result:
                alpha, cl, cd, cdp, cm, top_xtr, bot_xtr = result
                index = int(os.path.splitext(filename)[0])
                mach = machs[index]
                if alpha != alphas[index]: print("shit")
                data.append([alpha, mach, cl, cd, cdp, cm, top_xtr, bot_xtr])
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['Alpha', 'Mach', 'CL', 'CD', 'CDp', 'CM', 'Top_Xtr', 'Bot_Xtr'])
    return df

In [46]:
for alpha in ALPHAS:
    folder_path = f"{OUTPUT_FILE}/{alpha}"
    alphas = np.load(f"{folder_path}/alphas.npy")
    machs = np.load(f"{folder_path}/machs.npy")
    df = process_folder(machs, alphas, folder_path)
    df.to_csv(f"{folder_path}/filename.csv", index=True)